In [1]:
import numpy as np
import cv2
import glob

# 设置棋盘格的规格
chessboard_size = (10, 7)  # 棋盘格的内角点数量 (列, 行)
square_size = 0.07  # 棋盘格方格的大小，单位：米

# 准备对象点，棋盘格角点的三维坐标（假设棋盘格位于z=0平面）
obj_points = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
obj_points[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
obj_points *= square_size  # 将其单位转化为实际的物理尺寸（米）

# 用于存储检测到的角点
obj_points_list = []  # 三维世界坐标
img_points_left = []  # 左相机的二维图像坐标
img_points_right = []  # 右相机的二维图像坐标

# 加载左、右相机的标定图像
left_images = glob.glob('/home/jg/图片/双目/1223/cali_left/*.jpg')  # 假设左相机图像存放路径
right_images = glob.glob('/home/jg/图片/双目/1223/cali_right/*.jpg')  # 假设右相机图像存放路径

# 确保左右相机的图像一一对应
assert len(left_images) == len(right_images)

for left_img_path, right_img_path in zip(left_images, right_images):
    left_img = cv2.imread(left_img_path)
    right_img = cv2.imread(right_img_path)

    # 将图像转换为灰度图
    gray_left = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
    gray_right = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)

    # 查找棋盘格角点
    ret_left, corners_left = cv2.findChessboardCorners(gray_left, chessboard_size, None)
    ret_right, corners_right = cv2.findChessboardCorners(gray_right, chessboard_size, None)

    if ret_left and ret_right:
        obj_points_list.append(obj_points)
        img_points_left.append(corners_left)
        img_points_right.append(corners_right)

# 双目相机标定：计算旋转矩阵 R 和平移向量 T
ret, mtx_left, dist_left, mtx_right, dist_right, R, T, E, F = cv2.stereoCalibrate(
    obj_points_list, img_points_left, img_points_right,
    None, None, None, None, gray_left.shape[::-1], 
    flags=cv2.CALIB_FIX_INTRINSIC
)

print("旋转矩阵 R: \n", R)
print("平移向量 T: \n", T)

# 可选：保存标定结果
np.savez("stereo_calibration.npz", mtx_left=mtx_left, dist_left=dist_left,
         mtx_right=mtx_right, dist_right=dist_right, R=R, T=T)

旋转矩阵 R: 
 [[ 9.99988086e-01 -4.88135817e-03 -2.30800707e-05]
 [ 4.88135794e-03  9.99988086e-01 -9.70168024e-06]
 [ 2.31271531e-05  9.58890257e-06  1.00000000e+00]]
平移向量 T: 
 [[-1.45236357e-01]
 [-1.48300567e-02]
 [-8.68594517e-05]]


In [2]:
import numpy as np
import cv2
import glob

# 设置棋盘格的规格
chessboard_size = (10, 7)  # 棋盘格的内角点数量 (列, 行)
square_size = 0.07  # 棋盘格方格的大小，单位：米

# 准备对象点，棋盘格角点的三维坐标（假设棋盘格位于z=0平面）
obj_points = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
obj_points[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
obj_points *= square_size  # 将其单位转化为实际的物理尺寸（米）

# 用于存储检测到的角点
obj_points_list = []  # 三维世界坐标
img_points_left = []  # 左相机的二维图像坐标
img_points_right = []  # 右相机的二维图像坐标

# 加载左、右相机的标定图像
left_images = glob.glob('/home/jg/图片/双目/cali_left/*.jpg')  # 假设左相机图像存放路径
right_images = glob.glob('/home/jg/图片/双目/cali_right/*.jpg')  # 假设右相机图像存放路径

# 确保左右相机的图像一一对应
assert len(left_images) == len(right_images)

# 加载左右相机的相机内参（你应该在之前的标定中得到这些）
mtx_left = np.load('left_camera_matrix.npy')
dist_left = np.load('left_dist_coeffs.npy')
mtx_right = np.load('right_camera_matrix.npy')
dist_right = np.load('right_dist_coeffs.npy')

for left_img_path, right_img_path in zip(left_images, right_images):
    left_img = cv2.imread(left_img_path)
    right_img = cv2.imread(right_img_path)

    # 将图像转换为灰度图
    gray_left = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
    gray_right = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)

    # 查找棋盘格角点
    ret_left, corners_left = cv2.findChessboardCorners(gray_left, chessboard_size, None)
    ret_right, corners_right = cv2.findChessboardCorners(gray_right, chessboard_size, None)

    if ret_left and ret_right:
        obj_points_list.append(obj_points)
        img_points_left.append(corners_left)
        img_points_right.append(corners_right)

# 双目相机标定：计算旋转矩阵 R 和平移向量 T
ret, mtx_left, dist_left, mtx_right, dist_right, R, T, E, F = cv2.stereoCalibrate(
    obj_points_list, img_points_left, img_points_right,
    mtx_left, dist_left, mtx_right, dist_right,
    gray_left.shape[::-1], flags=cv2.CALIB_FIX_INTRINSIC
)

print("旋转矩阵 R: \n", R)
print("平移向量 T: \n", T)

# 可选：保存标定结果
np.savez("stereo_calibration_result.npz", mtx_left=mtx_left, dist_left=dist_left,
         mtx_right=mtx_right, dist_right=dist_right, R=R, T=T)

FileNotFoundError: [Errno 2] No such file or directory: 'left_camera_matrix.npy'